In [1]:
using Plots 
using CSV
using Tables
using DataFrames
using GLM
using Measures
using Statistics
plotly()

Plots.PlotlyBackend()

In [2]:
colors=palette(:seaborn_colorblind)

In [3]:
res = DataFrame(CSV.File("clinicalAndMetricsForUpload.csv"))

,Sample,Mutation_spectrum(C>A/C>G/C>T/T>A/T>C/T>G),Mutation_Count(lognumber)
,String31,String31,Int64
1,DLBCL-LS951,8/12/40/5/17/9,94
2,DLBCL-LS762,4/1/13/1/4/2,28
3,DLBCL-RICOVER_107,15/6/51/5/15/18,118
4,DLBCL-RICOVER_274,10/11/60/4/6/8,103
5,DLBCL-LS148,12/3/21/2/8/3,50
6,DLBCL-RICOVER_181,12/11/59/9/13/16,129
7,DLBCL-LS4619,9/11/54/12/14/11,115
8,DLBCL-LS88,8/7/8/0/3/2,31
9,DLBCL-RICOVER_1219,14/7/54/3/7/4,92


In [4]:
res.synMetric=res.peakSyn./res.peakEIA;
res.COONumber=res."COO_by_Any_GEP_+_Nanostring(string)"
res[res.COONumber.=="GCB",:COONumber].=string(1)
res[res.COONumber.=="ABC",:COONumber].=string(2)
res[res.COONumber.=="Unclassified",:COONumber].=string(3)
res[res.COONumber.=="N/A",:COONumber].=string(3)


res.OS=res."Overall_Survival_Months(number)";

In [5]:
# This is the last deceased patient before the first living one
lowOS_res=res[res.OS .<= 33.9, :];

In [6]:
pred_all = DataFrame(OS = 0:1:127);

In [7]:
function plotSyn(OS_col, metric_col, ylabel, n)    
        p = plot(legend=:outertopright)
            plot!(pred.OS, predFrame.prediction, label="model", linewidth=2,color=colors[1])
            plot!(pred.OS, predFrame.prediction, label="model", linewidth=0, color=colors[1],fillalpha=0.3,
                    ribbon = (predFrame.prediction .- predFrame.lower, predFrame.upper .- predFrame.prediction))
            plot!(OS_col, metric_col, label="data", seriestype=:scatter, xlab="Overall survival (months)",
                   ylab=ylabel, color=colors[4],markersize=5,markerstrokewidth=0)
            plot!(legend=false)
            #plot!(yticks=(log10.(collect(range(100,1000,10))),collect(range(100,1000,10))))
            if n == 130
                plot!(xticks=((collect(range(0,130,14))),collect(range(0,130,14))))
            else
                plot!(xticks=((collect(range(0,30,7))),collect(range(0,30,7))))
            end
            plot!(size=(900,500))
            #plot!(left_margin = 10mm)
            plot!(tickfontsize=12, xguidefontsize=18, yguidefontsize=18)
        display(p)
    return(p)
end

plotSyn (generic function with 1 method)

In [8]:
function plotSynCluster(OS_col, metric_col, ylabel, n,colorCol)    
        p = plot(legend=:outertopright)
            plot!(pred.OS, predFrame.prediction, label=false, linewidth=2,color=colors[1])
            plot!(pred.OS, predFrame.prediction, label=false, linewidth=0, color=colors[1],fillalpha=0.3,
                    ribbon = (predFrame.prediction .- predFrame.lower, predFrame.upper .- predFrame.prediction))
            plot!(OS_col, metric_col, label=colorCol, seriestype=:scatter, xlab="Overall survival (months)",
                   ylab=ylabel, color=colorCol,markersize=5,markerstrokewidth=0,palette =colors)
            plot!(legend=false)
            #plot!(yticks=(log10.(collect(range(100,1000,10))),collect(range(100,1000,10))))
            if n == 130
                plot!(xticks=((collect(range(0,130,14))),collect(range(0,130,14))))
            else
                plot!(xticks=((collect(range(0,30,7))),collect(range(0,30,7))))
            end
            plot!(size=(900,500))
            #plot!(left_margin = 10mm)
            plot!(tickfontsize=12, xguidefontsize=18, yguidefontsize=18)
        display(p)
    return(p)
end

plotSynCluster (generic function with 1 method)

## Low overall survival results - synergy metric

In [9]:
### OS(M) vs synergy 
pred = DataFrame(OS = 0:1:34);
model = lm(@formula(synMetric ~ OS), lowOS_res)
predFrame = predict(model, pred, level = 0.95,interval = :confidence);
plotSyn(lowOS_res."Overall_Survival_Months(number)", lowOS_res.synMetric, "Synergy score", 34)
println("r squared = "*string(r2(model)))

r squared = 0.21216333789438935


In [ ]:
pred = DataFrame(OS = 0:1:34);
model = lm(@formula(synMetric ~ OS), lowOS_res)
predFrame = predict(model, pred, level = 0.95,interval = :confidence);
plotSynCluster(lowOS_res."Overall_Survival_Months(number)", lowOS_res.synMetric, "Synergy score", 30,lowOS_res."Cluster(number)")
println("r squared = "*string(r2(model)))

In [ ]:
pred = DataFrame(OS = 0:1:34);
model = lm(@formula(synMetric ~ OS), lowOS_res)
predFrame = predict(model, pred, level = 0.95,interval = :confidence);
plotSynCluster(lowOS_res."Overall_Survival_Months(number)", lowOS_res.synMetric, "Synergy score", 30,parse.(Int64, lowOS_res.COONumber))
println("r squared = "*string(r2(model)))

In [ ]:
pred = DataFrame(OS = 0:1:34);
model = lm(@formula(synMetric ~ OS), lowOS_res)
predFrame = predict(model, pred, level = 0.95,interval = :confidence);
plotSynCluster(lowOS_res."Overall_Survival_Months(number)", lowOS_res.synMetric, "Synergy score", 30,parse.(Int64, lowOS_res."IPI(number)"))
println("r squared = "*string(r2(model)))

# using peak cell number

In [ ]:
pred = DataFrame(OS = 0:1:34);
model = lm(@formula(peakCN~ OS), lowOS_res)
predFrame = predict(model, pred, level = 0.95,interval = :confidence);
plotSyn(lowOS_res."Overall_Survival_Months(number)", lowOS_res.peakCN, "peak CN ", 30)
println("r squared = "*string(r2(model)))

In [ ]:
lowOS_res.log10peakCN=log10.(lowOS_res.peakCN)
pred = DataFrame(OS = 0:1:34);
model = lm(@formula(log10peakCN~ OS), lowOS_res)
predFrame = predict(model, pred, level = 0.95,interval = :confidence);
plotSyn(lowOS_res."Overall_Survival_Months(number)", lowOS_res.log10peakCN, "log10 peak CN", 30)
println("r squared = "*string(r2(model)))